<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/ch16_integrationTechniques_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 16 (Integration techniques)

---

# About this code file:

### This notebook contains full code solutions to the exercises in this book chapter. There are many correct ways to solve the exercises; this notebook provides *a* solution, not *THE* solution. Please use this code as a starting point to continue exploring and experimenting with calculus concepts and visualizations.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import scipy.integrate as spi
import matplotlib.pyplot as plt
from IPython.display import Math

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Exercise 16.1-2: Integration by parts in sympy

In [ ]:
x,C = sym.symbols('x,C')

fx = x*sym.cos(x)
# fx = sym.exp(-x*sym.pi)*sym.sin(7*x) # for exercise 2
antideriv = sym.integrate(fx)

# show the result using latex
display(Math('%s = %s+C' %(sym.latex(sym.Integral(fx)),sym.latex(antideriv))))

In [ ]:
# discretize the function
xx = np.linspace(-2*np.pi,3*np.pi,803)
y = [fx.subs(x,xi) for xi in xx]
F = [antideriv.subs(x,xi) for xi in xx]

# and make some lovely plots :)
_,axs = plt.subplots(1,2,figsize=(10,4))

axs[0].plot(y,F,'k',linewidth=4)
axs[0].set(xlabel='f(x)',ylabel='F(x)')
axs[1].scatter(y,F,s=30,c=xx,cmap='gist_ncar') # FYI: https://matplotlib.org/stable/users/explain/colors/colormaps.html
axs[1].axis('off')

plt.tight_layout()
plt.savefig('intTech_ex1.png')
plt.show()

# Exercise 16.3

In [ ]:
x,A,B = sym.symbols('x,A,B')

# function
fx = (5*x+3) / (2*x**2 - 4*x - 6)

# quickie-plot
sym.plot(fx,(x,-5,5),ylim=[-20,20])
plt.show()

display(Math('f(x) = %s' %sym.latex(fx)))
print('')
display(Math('\int f(x) \,dx = \,?'))

In [ ]:
# Step 1: separate numerator and denominator
numerator, denominator = fx.as_numer_denom()

# and print
display(Math('\\text{The numerator is } %s' %sym.latex(numerator)))
display(Math('\\text{The denominator is } %s' %sym.latex(denominator)))

In [ ]:
# Step 2: factor the denominator
den_factors = sym.factor(denominator)

# print them out
for i,fact in enumerate(den_factors.args):
  display(Math('\\text{Demoninator factor } %g: \; %s' %(i+1,sym.latex(fact))))
  print('')

In [ ]:
# Step 3: create simple fractions
simple_fract_1 = A / (den_factors.args[0]*den_factors.args[1])
simple_fract_2 = B / den_factors.args[2]

display(Math('\\text{Simple fraction 1:} \; %s' %sym.latex(simple_fract_1)))
print('')
display(Math('\\text{Simple fraction 2:} \; %s' %sym.latex(simple_fract_2)))

In [ ]:
# Step 4: solve for A and B
expression = sym.Eq(numerator , simple_fract_1*sym.prod(den_factors.args) + simple_fract_2*sym.prod(den_factors.args) )
solutionsAB = sym.solve(expression,(A,B))

solutionsAB

In [ ]:
# Step 5: integrate separately
intPart1 = sym.integrate( simple_fract_1.subs(A,solutionsAB[A]) )
intPart2 = sym.integrate( simple_fract_2.subs(B,solutionsAB[B]) )

display(Math('\int %s \,dx = %s+C' %(sym.latex(simple_fract_1.subs(A,solutionsAB[A])),sym.latex(intPart1))))
print('')
display(Math('\int %s \,dx = %s+C' %(sym.latex(simple_fract_2.subs(B,solutionsAB[B])),sym.latex(intPart2))))

In [ ]:
# Step 6: sum the parts
mysolution = intPart1 + intPart2
mysolution

display(Math('\\int %s \, dx = %s+C' %(sym.latex(fx),sym.latex(mysolution))))

# Exercise 16.4: The lemon and the infinite

In [ ]:
# functions
x = sym.symbols('x')
f = 10*sym.sin( sym.sin(sym.sin(x)**1)**3 )**5
g =    sym.cos( sym.cos(sym.cos(x)**2)**4 )**6

# derivatives
df = sym.diff(f,x)
dg = sym.diff(g,x)

# lambdify
f_lam  = sym.lambdify(x,f,'numpy')
g_lam  = sym.lambdify(x,g,'numpy')
df_lam = sym.lambdify(x,df,'numpy')
dg_lam = sym.lambdify(x,dg,'numpy')

# integrals
xx = np.linspace(0,5*np.pi,5001)
f_int = spi.cumulative_simpson(f_lam(xx),x=xx,initial=0)
g_int = spi.cumulative_simpson(g_lam(xx),x=xx,initial=0)


# and plot
_,axs = plt.subplots(2,2,figsize=(14,8))
axs[0,0].plot(xx,f_lam(xx),'k',label='$f(x)$')
axs[0,0].plot(xx,g_lam(xx),'--',color=[.7,.7,.7],label='$g(x)$')
axs[0,0].legend()
axs[0,0].set(xlabel='$x$',ylabel='$y = f(x)$ or $g(x)$',title=r'$\bf{A}$)  Functions',xlim=xx[[0,-1]])

axs[0,1].plot(f_lam(xx), df_lam(xx),'k',label='$f_1$')
axs[0,1].plot(g_lam(xx), dg_lam(xx),'--',color=[.7,.7,.7],label='$f_1$')
axs[0,1].set(xlabel='$f(x)$ or $g(x)$',ylabel="$f\,'(x)$ or $g\,'(x)$",title=r'$\bf{B}$)  Derivatives by their functions')

axs[1,0].plot(xx, f_int,'k',label='$F(x)$')
axs[1,0].plot(xx, g_int,'--',color=[.7,.7,.7],label='$G(x)$')
axs[1,0].legend()
axs[1,0].set(xlabel='$x$',ylabel='$Y = F(x)$ or $G(x)$',title=r'$\bf{C}$)  Antiderivatives by $x$',xlim=xx[[0,-1]])

axs[1,1].plot(f_int,g_int,'k')
axs[1,1].set(xlabel='$F(x)$',ylabel='$G(x)$',title=r'$\bf{D}$)  Antiderivatives with each other')


plt.tight_layout()
plt.savefig('intTech_ex4.png')
plt.show()